In [7]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
import os


In [8]:
def get_input_files(base_path):
    """
    Recursively get all files from league folders
    Returns a dictionary: {league_name: [file_paths]}
    """
    league_file_map = {}

    # Get all league folders
    league_folders = [f for f in os.listdir(base_path)
                      if os.path.isdir(os.path.join(base_path, f)) and not f.startswith('.')]

    for league in league_folders:
        league_path = os.path.join(base_path, league)
        input_files = []

        for root, _, files in os.walk(league_path):
            for file in files:
                if file.endswith(('.xlsx', '.csv')):  # only accept Excel or CSV files
                    input_files.append(os.path.join(root, file))
        
        league_file_map[league] = input_files

    return league_file_map

In [9]:
# List of input CSV files
input_files = get_input_files("big-5-scouting-data")
print(input_files)

{'Serie_A_Scouting_Data': ['big-5-scouting-data/Serie_A_Scouting_Data/Atalanta_Scouting_Data.xlsx', 'big-5-scouting-data/Serie_A_Scouting_Data/Hellas_Verona_Scouting_Data.xlsx', 'big-5-scouting-data/Serie_A_Scouting_Data/Torino_Scouting_Data.xlsx', 'big-5-scouting-data/Serie_A_Scouting_Data/Como_Scouting_Data.xlsx', 'big-5-scouting-data/Serie_A_Scouting_Data/Roma_Scouting_Data.xlsx', 'big-5-scouting-data/Serie_A_Scouting_Data/Udinese_Scouting_Data.xlsx', 'big-5-scouting-data/Serie_A_Scouting_Data/Milan_Scouting_Data.xlsx', 'big-5-scouting-data/Serie_A_Scouting_Data/Bologna_Scouting_Data.xlsx', 'big-5-scouting-data/Serie_A_Scouting_Data/Parma_Scouting_Data.xlsx', 'big-5-scouting-data/Serie_A_Scouting_Data/Napoli_Scouting_Data.xlsx', 'big-5-scouting-data/Serie_A_Scouting_Data/Venezia_Scouting_Data.xlsx', 'big-5-scouting-data/Serie_A_Scouting_Data/Fiorentina_Scouting_Data.xlsx', 'big-5-scouting-data/Serie_A_Scouting_Data/Cagliari_Scouting_Data.xlsx', 'big-5-scouting-data/Serie_A_Scouting_

In [10]:
def create_pivot_table(df):
    """
    Creates a pivot table from player statistics and flattens column names
    """
    pivot_df = df.pivot_table(
        index=["Player", "Versus", "BasedOnMinutes", "scouting_period"],
        columns=["Statistic"],
        values=["Per90", "Percentile"]
    ).reset_index()

    pivot_df.columns = [
        '_'.join(str(i) for i in col if str(i) != '')
        if isinstance(col, tuple) else str(col)
        for col in pivot_df.columns
    ]
    return pivot_df

In [12]:
# Path to base folder
base_path = "big-5-scouting-data"

# Get all files grouped by league
league_file_map = get_input_files(base_path)

# Make sure output directory exists
os.makedirs("combined_data", exist_ok=True)

# Process each league
for league, files in league_file_map.items():
    df_list = []

    for file in files:
        try:
            if file.endswith('.xlsx'):
                df = pd.read_excel(file)
            elif file.endswith('.csv'):
                df = pd.read_csv(file)
            df_list.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

    if not df_list:
        print(f"No files found for {league}")
        continue

    combined_df = pd.concat(df_list, ignore_index=True)
    outfield_df = combined_df[combined_df['Versus'] != 'Goalkeepers']

    if not outfield_df.empty:
        pivot_outfield_df = create_pivot_table(outfield_df)
        output_file = os.path.join("combined_data", f"{league}_outfield_stats.csv")
        pivot_outfield_df.to_csv(output_file, index=False)
        print(f"Saved: {output_file}")
    else:
        print(f"No outfield data for {league}")

Saved: combined_data/Serie_A_Scouting_Data_outfield_stats.csv
Saved: combined_data/Premier_League_Data_outfield_stats.csv
Saved: combined_data/Ligue_1_Scouting_Data_outfield_stats.csv
Saved: combined_data/Bundesliga_Scouting_Data_outfield_stats.csv
Saved: combined_data/La_Liga_Scouting_Data_outfield_stats.csv


In [6]:
gk_df = pd.read_csv("combined_data/goalkeeper_stats.csv")
outfield_df = pd.read_csv("combined_data/outfield_stats.csv")
print(f"Number of goalkeeper records: {len(gk_df)}")
print(f"Number of outfield player records: {len(outfield_df)}")

Number of goalkeeper records: 1303
Number of outfield player records: 14696


In [7]:
gk_df.head()

,Player,Versus,BasedOnMinutes,scouting_period,Per90_Average Pass Length,Per90_Avg. Distance of Def. Actions,Per90_Avg. Length of Goal Kicks,Per90_Clean Sheet Percentage,Per90_Clean Sheets,Per90_Corner Kick Goals Against,...,Percentile_Penalty Kicks Attempted,Percentile_Penalty Kicks Missed,Percentile_Penalty Kicks Saved,Percentile_Post-Shot Expected Goals,Percentile_Save Percentage,Percentile_Save% (Penalty Kicks),Percentile_Saves,Percentile_Shots on Target Against,Percentile_Throws Attempted,Percentile_Wins
0,Aaron Ramsdale,Goalkeepers,300,2022-2023 Europa League,38.2,17.5,65.0,33.3,0.33,0.33,...,21.0,49.0,46.0,79.0,28.0,NaN,13.0,16.0,48.0,45.0
1,Aaron Ramsdale,Goalkeepers,540,2023-2024 Premier League,33.3,14.6,41.4,33.3,0.33,0.17,...,6.0,45.0,41.0,98.0,23.0,NaN,5.0,5.0,63.0,98.0
2,Aaron Ramsdale,Goalkeepers,1260,2024-2025 Premier League,33.5,12.2,45.3,14.3,0.14,0.50,...,66.0,50.0,38.0,9.0,41.0,38.0,95.0,98.0,77.0,9.0
3,Aaron Ramsdale,Goalkeepers,1350,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",34.4,11.4,44.8,13.3,0.13,0.47,...,44.0,40.0,27.0,1.0,39.0,27.0,98.0,98.0,81.0,5.0
4,Aaron Ramsdale,Goalkeepers,3060,2021-2022 Premier League,33.2,16.4,55.1,35.3,0.35,0.09,...,73.0,90.0,37.0,71.0,75.0,37.0,33.0,21.0,63.0,90.0


In [8]:
outfield_df.head()

,Player,Versus,BasedOnMinutes,scouting_period,Per90_% of Aerials Won,Per90_% of Dribblers Tackled,Per90_Aerials Lost,Per90_Aerials Won,Per90_Assists,Per90_Average Shot Distance,...,Percentile_Touches (Def Pen),Percentile_Touches (Live-Ball),Percentile_Touches (Mid 3rd),Percentile_Yellow Cards,Percentile_npxG + xAG,Percentile_npxG/Shot,Percentile_npxG: Non-Penalty xG,Percentile_xA: Expected Assists,Percentile_xAG: Exp. Assisted Goals,Percentile_xG: Expected Goals
0,Aaron Cresswell,Center Backs,3069,2017-2018 Premier League,54.8,63.3,1.38,1.67,0.21,28.7,...,3.0,64.0,63.0,31.0,80.0,8.0,23.0,92.0,96.0,23.0
1,Aaron Cresswell,Fullbacks,416,2023-2024 Europa League,50.0,66.7,1.30,1.30,0.00,15.5,...,37.0,67.0,69.0,46.0,26.0,78.0,36.0,47.0,31.0,35.0
2,Aaron Cresswell,Fullbacks,517,2022-2023 Europa Conference League,22.2,80.0,1.22,0.35,0.35,25.0,...,39.0,77.0,90.0,20.0,49.0,24.0,12.0,65.0,67.0,12.0
3,Aaron Cresswell,Fullbacks,664,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",52.9,83.3,1.08,1.22,0.00,23.6,...,66.0,49.0,41.0,25.0,32.0,72.0,32.0,45.0,45.0,32.0
4,Aaron Cresswell,Fullbacks,696,2021-2022 Europa League,55.6,44.4,0.52,0.65,0.26,20.1,...,34.0,89.0,76.0,58.0,95.0,50.0,26.0,99.0,99.0,25.0
